## 深度学习八股基础

2025-10-11 18:34:53 Saturday



### 1.请你从直观上解释一下 交叉熵（Cross Entropy） 的含义，它为什么能作为分类任务的损失函数？


交叉熵衡量预测分布与真实分布的差异，定义为 $H(p,q)=-\sum p_i\log q_i$.

对于分类任务，真实分布是 one-hot，所以它只取真实类别的预测概率加入loss计算，即$H(p,q)=-\sum \log q_i$.。

交叉熵最小时，说明预测分布最接近真实分布，相当于最大化真实类别的概率，也就是最大似然估计。”


#### 延伸：交叉熵为什么比 MSE 更适合分类?
相比均方误差，交叉熵在分类任务中对概率差异更敏感，梯度更大，收敛更快。

### 2.请你解释一下 Batch Normalization（BN） 的作用原理，以及它为什么能加快收敛、改善训练稳定性？

Batch Normalization（BN）主要用于对网络中间层的特征进行归一化处理，从而提高训练稳定性并加快收敛速度。
如果不进行归一化，不同样本批次（batch）之间的输入分布可能差异很大，导致梯度方向不稳定，训练过程容易震荡或发散。BN 的核心思想是：**在每个batch 内计算特征的均值与方差，对输入进行标准化，再通过可学习的线性变换恢复适当的数值范围。**

其计算过程如下：

$$
\mu_B = \frac{1}{m} \sum_{i=1}^{m} x_i
$$

$$
\sigma_B^2 = \frac{1}{m} \sum_{i=1}^{m} (x_i - \mu_B)^2
$$

归一化后得到标准化特征：
$$
\hat{x}_i = \frac{x_i - \mu_B}{\sqrt{\sigma_B^2 + \epsilon}}
$$

再通过可学习的缩放和平移参数（$\gamma$, $\beta$）进行线性变换：
$$
y_i = \gamma \hat{x}_i + \beta
$$


这样可以迫使经过BN后的输出分布对于不同batch也是一致的（即输出的均值和方差由可学习线形层决定）。

$$
\mu_B = \frac{1}{m}\sum x_i
$$, 

在 NLP 任务中，由于一个 batch 内样本（序列）的长度通常不一致，因此更常使用 **Layer Normalization（LN）**。LN 是对**单个样本内部的所有特征维度**进行归一化，而非对整个 batch，从而更适合处理变长序列。



### 3.请你讲讲 **Adam 优化器** 的原理，为什么要引入一阶与二阶矩估计？

- 过去方法：
    - Adam之前是SGD，SGD是只用当前批次的梯度进行更新，存在两个问题 是容易在局部最优附近震荡 和学习率固定；
        公式为$$\theta_{t+1} = \theta_t - \alpha \cdot g_t$$
    - 对于震荡问题，Momentum提出通过累积历史梯度的“移动平均”来平滑梯度。公式为：
        $$m_t = \beta \cdot m_{t-1} + (1 - \beta)\cdot g_t, \\ \theta_{t+1} = \theta_t - \alpha \cdot m_t$$


    - 对于学习率固定，RMSProp 通过对梯度平方取指数滑动平均，抑制梯度剧烈波动，实现每个参数的自适应学习率，但缺少动量项，会在梯度方向频繁变化时收敛不稳。公式为： 
        $$ v_t = \beta \cdot v_{t-1} + (1 - \beta) \cdot g_t^2, \\ \theta_{t+1} = \theta_t - \alpha \cdot \frac{g_t}{\sqrt{v_t} + \epsilon} $$


- Adam结合这两个做改进，

    一阶矩估计（$m_t$）：梯度的指数滑动平均，起到动量作用，平滑震荡；

    二阶矩估计（$v_t$）：梯度平方的滑动平均，用于自适应调整学习率。
    
    公式如下：
    $$m_t = \beta_1 m_{t-1} + (1-\beta_1) g_t$$
    $$v_t = \beta_2 v_{t-1} + (1-\beta_2) g_t^2$$
    由于初始阶段偏向 0，需要做偏差校正：
    $$\hat{m}_t = \frac{m_t}{1 - \beta_1^t},\quad \hat{v}_t = \frac{v_t}{1 - \beta_2^t}$$

    最终更新参数：
    $$\theta_{t+1} = \theta_t - \alpha \frac{\hat{m}_t}{\sqrt{\hat{v}_t} + \epsilon}$$

    故而一阶平滑方向，二阶稳定步长，故而Adam比SDG在梯度噪声大更稳定收敛。
    而本质上Adam是自动调节步长，在梯度变化剧烈（$g_t^2$ 大）时调小步长，增加稳定性；在梯度变化微弱($g_t$小)时调大步长加速收敛。

    若某个参数梯度方差大，则 $v_t$ 大 → 步长小；反之步长大。这就是它能在稀疏特征任务中表现好的根本原因。


#### 延伸：为什么需要除以 $(1-\beta_1^t)$？它的意义是什么？
因为在训练早期 $m_t$、$v_t$ 的初值是 0，滑动平均在最开始几步被拉低，偏向 0。除以 $(1-\beta_1^t)$ 和 $(1-\beta_2^t)$ 可以补偿这个偏差，使得早期估计更准确。



### 4.请解释一下 梯度消失（Vanishing Gradient） 的原因和常见解决方法。

梯度消失是说在反向传播的过程中，前层梯度接近为0，而几乎得不到更新。


分析中间过程，那么梯度为0的原因可能有点，
一是网络过深，因为梯度更新是链式乘法，解决的方法可以引入残差网络，使得部分梯度可以不用走那么多层，
二是激活函数包含。如 Sigmoid、tanh 在输入过大或过小时导数几乎为 0，导致梯度乘积趋 0，这时候要么换不饱和的激活函数，要么就是前面加归一化层。


### 5.请你讲讲 Transformer 的自注意力机制（Self-Attention） 是怎么计算的？

输入x为batchszie*seqlength*hidden_dim, 经过线形层分别得到q，k，v，然后q与k做点积，表示计算相关性；除以 $\sqrt{d_k}$ 是为了防止点积值过大导致 进入softmax 的饱和区。
再做softmax后得到的权重矩阵表示每个 token 对序列中其他 token 的注意力分布，使用softmax而不用其他归一化方法是因为它能加强重点，削弱非重点；最后再与v计算，相当于对所有位置加权求和，以实现全局注意力。
常用的多头是是把hidden_dim变成了好几个头，每个头有独立的q,k,v去捕捉不同的注意模式。